In [36]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import numpy as np

# Data Manipulation
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# stacking
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve, auc, precision_score, recall_score, f1_score

# imbalance
from imblearn.over_sampling import SMOTE

In [37]:
df = pd.read_excel("C:\\Users\\Anthony\\Pictures\\ml\\supervised-learning\\cross-sell-recommendation\\data\\january.xlsx")
df.head()

,Customers,A/C,Date,P,S,B,200g,O.P.,800g,1500g,ST,600g,KATIKATI,MANDAZI,DOUGHNUTS,50-50 SLICED,MANDO FAMILY PACK,4 Square,Qty
0,YUSSUF HUSSEIN YARE - NEW NAROK DEPOT (2294),2294,2026-01-07,0,390,105,96,0,18,0,0,84,0,20,0,0,0,0,713
1,WOTE DEPOT (1817),1817,2026-01-01,0,4365,2010,672,525,0,0,0,273,0,10,0,30,20,0,7905
2,WOOLMART SUPERMARTKET - NAKURU EAST (1579),1579,2026-01-02,0,435,435,699,144,296,0,0,752,0,0,0,7,0,134,2902
3,WOOLMART SUPERMARTKET - NAKURU CENTRAL (1578),1578,2026-01-02,0,416,259,464,39,301,0,0,546,0,0,0,29,0,31,2085
4,WESTERN MART -NGARA (1293),1293,2026-01-01,0,246,220,453,10,88,0,0,250,0,48,46,15,0,0,1376


In [38]:
products = [
    '200g','S','B','600g','4 Square','800g','P',
    'MANDAZI','50-50 SLICED','DOUGHNUTS','MANDO FAMILY PACK'
]

binary_df = df.copy()

for col in products:
    binary_df[col] = (binary_df[col] > 0).astype(int)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 805 entries, 0 to 804
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Customers          805 non-null    object        
 1   A/C                805 non-null    int64         
 2   Date               805 non-null    datetime64[ns]
 3   P                  805 non-null    int64         
 4   S                  805 non-null    int64         
 5   B                  805 non-null    int64         
 6   200g               805 non-null    int64         
 7   O.P.               805 non-null    int64         
 8   800g               805 non-null    int64         
 9   1500g              805 non-null    int64         
 10  ST                 805 non-null    int64         
 11  600g               805 non-null    int64         
 12  KATIKATI           805 non-null    int64         
 13  MANDAZI            805 non-null    int64         
 14  DOUGHNUTS 

In [40]:
df.isna().sum()

Customers            0
A/C                  0
Date                 0
P                    0
S                    0
B                    0
200g                 0
O.P.                 0
800g                 0
1500g                0
ST                   0
600g                 0
KATIKATI             0
MANDAZI              0
DOUGHNUTS            0
50-50 SLICED         0
MANDO FAMILY PACK    0
4 Square             0
Qty                  0
dtype: int64

In [41]:
df.isna().sum()

Customers            0
A/C                  0
Date                 0
P                    0
S                    0
B                    0
200g                 0
O.P.                 0
800g                 0
1500g                0
ST                   0
600g                 0
KATIKATI             0
MANDAZI              0
DOUGHNUTS            0
50-50 SLICED         0
MANDO FAMILY PACK    0
4 Square             0
Qty                  0
dtype: int64

In [42]:
df.columns

Index(['Customers', 'A/C', 'Date', 'P', 'S', 'B', '200g', 'O.P.', '800g',
       '1500g', 'ST', '600g', 'KATIKATI', 'MANDAZI', 'DOUGHNUTS',
       '50-50 SLICED', 'MANDO FAMILY PACK', '4 Square', 'Qty'],
      dtype='object')

In [43]:
df.shape

(805, 19)

In [44]:
df.describe()

,A/C,Date,P,S,B,200g,O.P.,800g,1500g,ST,600g,KATIKATI,MANDAZI,DOUGHNUTS,50-50 SLICED,MANDO FAMILY PACK,4 Square,Qty
count,805.000000,805,805.000000,805.000000,805.000000,805.000000,805.000000,805.000000,805.0,805.0,805.000000,805.0,805.000000,805.000000,805.000000,805.000000,805.000000,805.000000
mean,2141.855901,2026-01-01 13:23:10.807453440,126.320497,3381.437267,1350.181366,3716.443478,323.186335,224.567702,0.0,0.0,924.808696,0.0,50.237267,15.708075,20.939130,6.237267,242.863354,10382.930435
min,55.000000,2026-01-01 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000
25%,1300.000000,2026-01-01 00:00:00,0.000000,411.000000,255.000000,0.000000,0.000000,0.000000,0.0,0.0,234.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1794.000000
50%,1793.000000,2026-01-01 00:00:00,0.000000,1020.000000,510.000000,432.000000,95.000000,145.000000,0.0,0.0,601.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,3987.000000
75%,2195.000000,2026-01-01 00:00:00,0.000000,3990.000000,1590.000000,4272.000000,285.000000,300.000000,0.0,0.0,1226.000000,0.0,48.000000,2.000000,1.000000,0.000000,120.000000,12648.000000
max,7098.000000,2026-01-29 00:00:00,15150.000000,41880.000000,16965.000000,73992.000000,9105.000000,6018.000000,0.0,0.0,9166.000000,0.0,9536.000000,538.000000,1243.000000,356.000000,20415.000000,130004.000000
std,1554.885038,NaN,740.855366,5545.694063,2092.014681,7931.431042,766.005565,370.094037,0.0,0.0,1135.156745,0.0,347.509973,44.782593,62.640251,25.883341,1097.892862,15991.097539


In [45]:
# conver the 'Customers' column to numeric, coercing errors to NaN
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
binary_df['Customers'] = le.fit_transform(binary_df['Customers'])

In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42),
    "Naive Bayes": GaussianNB(),
    "SVM": SVC(probability=True),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    "LightGBM": LGBMClassifier(),
    "CatBoost": CatBoostClassifier(verbose=0)
}


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.base import clone

best_models = {}

for product in products:
    X = binary_df[['Customers'] + [p for p in products if p != product]]
    y = binary_df[product]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y)

    best_auc = 0
    best_model = None

    for name, model in models.items():

        model_copy = clone(model)
        model_copy.fit(X_train, y_train)

        probs = model_copy.predict_proba(X_test)[:,1]
        auc = roc_auc_score(y_test, probs)

        if auc > best_auc:
            best_auc = auc
            best_model = model_copy

    best_models[product] = best_model




c:\Users\Anthony\Pictures\ml\venv\Lib\site-packages\xgboost\training.py:199: UserWarning: [16:18:15] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 472, number of negative: 172
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000379 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 231
[LightGBM] [Info] Number of data points in the train set: 644, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.732919 -> initscore=1.009485
[LightGBM] [Info] Start training from score 1.009485
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

c:\Users\Anthony\Pictures\ml\venv\Lib\site-packages\xgboost\training.py:199: UserWarning: [16:18:18] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 635, number of negative: 9
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000088 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 233
[LightGBM] [Info] Number of data points in the train set: 644, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.986025 -> initscore=4.256400
[LightGBM] [Info] Start training from score 4.256400
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -in

c:\Users\Anthony\Pictures\ml\venv\Lib\site-packages\xgboost\training.py:199: UserWarning: [16:18:20] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


...

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 162, number of negative: 482
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000080 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 231
[LightGBM] [Info] Number of data points in the train set: 644, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.251553 -> initscore=-1.090348
[LightGBM] [Info] Start training from score -1.090348
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

c:\Users\Anthony\Pictures\ml\venv\Lib\site-packages\xgboost\training.py:199: UserWarning: [16:18:30] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 162, number of negative: 482
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000089 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 231
[LightGBM] [Info] Number of data points in the train set: 644, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.251553 -> initscore=-1.090348
[LightGBM] [Info] Start training from score -1.090348
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

c:\Users\Anthony\Pictures\ml\venv\Lib\site-packages\xgboost\training.py:199: UserWarning: [16:18:32] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 78, number of negative: 566
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000110 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 231
[LightGBM] [Info] Number of data points in the train set: 644, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.121118 -> initscore=-1.981885
[LightGBM] [Info] Start training from score -1.981885
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

In [48]:
def recommend_products(customer_row, top_n=3):

    recommendations = {}

    for product, model in best_models.items():

        X_input = customer_row[['Customers'] + [p for p in products if p != product]]

        prob = model.predict_proba(X_input)[0][1]
        recommendations[product] = prob

    rec_df = pd.DataFrame(recommendations.items(),
                          columns=['Product','Probability'])

    # Remove already purchased
    bought = [p for p in products if customer_row[p].values[0] == 1]

    rec_df = rec_df[~rec_df['Product'].isin(bought)]

    rec_df = rec_df.sort_values(by='Probability', ascending=False)

    return rec_df.head(top_n)


In [50]:
# retrive customer namefrom the original dataframe and get recommendations
customer_example = binary_df.iloc[[0]]
customer_name = df.iloc[0]['Customers']  # assuming 'Customers' is the column with customer names

print(f"Recommendations for customer: {customer_name}")

recommend_products(customer_example, top_n=3)

Recommendations for customer: YUSSUF HUSSEIN YARE - NEW NAROK DEPOT (2294)


,Product,Probability
8,50-50 SLICED,0.229035
10,MANDO FAMILY PACK,0.220000
4,4 Square,0.195158
